In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
import urllib.parse

In [2]:
def get_random_page_urls(driver, languages: list) -> dict:
    """
    Récupère les URLs de pages aléatoires pour chaque langue donnée

    Parameters:
    driver (WebDriver): Le driver Selenium
    languages (list): La liste des langues pour lesquelles on veut récupérer une URL

    Returns:
    dict: Un dictionnaire avec les langues en clés et les URLs en valeurs
    """
    urls = {}
    for language in languages:
        driver.get('https://{}.wikipedia.org/wiki/Special:Random'.format(language))
        urls[language] = driver.current_url
    return urls

Je commence par définir une fonction nommée `get_random_page_urls` qui prend deux paramètres : `driver` et `languages`. Le `driver` est probablement un objet qui me permet d'interagir avec un navigateur web, et `languages` est une liste des codes de langue pour lesquels je veux récupérer des URL aléatoires de pages Wikipédia.

À l'intérieur de la fonction, je crée un dictionnaire vide appelé `urls` où je vais stocker les URL que je récupère. Pour chaque langue présente dans la liste `languages`, je fais plusieurs actions :

1. Je construis l'URL de la page Wikipédia aléatoire pour cette langue en utilisant la méthode `format`. Cela me donne une URL comme 'https://en.wikipedia.org/wiki/Special:Random' si la langue est 'en' (anglais), par exemple.

2. J'utilise le `driver` pour charger cette URL dans le navigateur web avec `driver.get(url_formée)`.

3. Une fois que la page a chargé, je récupère l'URL actuelle, qui est une page spécifique choisie aléatoirement par Wikipédia, en utilisant `driver.current_url`.

4. Je stocke cette URL dans le dictionnaire `urls` avec la langue correspondante comme clé.

Après avoir traité toutes les langues dans la boucle, je retourne le dictionnaire `urls` qui contient les codes de langue comme clés et les URL des pages Wikipédia aléatoires comme valeurs.

In [3]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def get_interlanguage_urls(driver, base_url: str) -> dict:
    """
    Obtient les URLs des pages correspondantes dans d'autres langues pour une page donnée

    Parameters:
    driver (WebDriver): Le driver Selenium
    base_url (str): L'URL de la page pour laquelle on veut récupérer les URLs des pages correspondantes dans d'autres langues

    Returns:
    dict: Un dictionnaire avec les langues en clés et les URLs en valeurs
    """
    driver.get(base_url)
    wait = WebDriverWait(driver, 30)
    try:
        interlanguage_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.interlanguage-link a')))
        return {link.get_attribute('lang'): link.get_attribute('href') for link in interlanguage_links}
    except TimeoutException:
        logging.error(f"Timeout when trying to find interlanguage links for {base_url}\n")
        return {}

Je commence par importer le module `logging` qui me permet d'enregistrer des informations sur ce qui se passe dans mon code. Je configure le niveau de logging à `INFO`, ce qui signifie que je vais enregistrer les messages de niveau INFO et supérieur (comme ERROR). Je définis également un format spécifique pour ces messages qui inclut la date et l'heure, le niveau de gravité du message et le message lui-même.

Ensuite, je définis une fonction nommée `get_interlanguage_urls` qui prend deux paramètres : `driver` et `base_url`. Le `driver` est l'objet qui interagit avec le navigateur, et `base_url` est l'URL de la page de base à partir de laquelle je veux récupérer des liens interlangues.

À l'intérieur de la fonction, je fais plusieurs actions :

1. Je charge l'URL de base dans le navigateur en utilisant `driver.get(base_url)`.

2. Je crée un objet `wait` de type `WebDriverWait`, lié au `driver` et avec un délai maximal de 30 secondes. Cet objet me permet d'attendre que certaines conditions soient remplies avant de continuer l'exécution de mon code.

3. J'essaie ensuite de récupérer tous les éléments de la page qui correspondent à des liens interlangues. Ces liens sont localisés par un sélecteur CSS qui cible les éléments 'a' dans les 'li' avec la classe 'interlanguage-link'. J'attends que ces éléments soient présents sur la page, ce que je vérifie avec la méthode `presence_of_all_elements_located`.

4. Si les éléments sont trouvés à temps, je construis un dictionnaire avec les langues comme clés (en récupérant l'attribut 'lang' de chaque lien) et les URL correspondantes comme valeurs (en récupérant l'attribut 'href' de chaque lien).

5. Si le délai d'attente de 30 secondes est dépassé sans que les liens soient trouvés, je consigne une erreur de type `TimeoutException` avec un message d'erreur spécifique qui mentionne l'URL de base. Dans ce cas, je retourne un dictionnaire vide.

La fonction `get_interlanguage_urls` retourne donc soit un dictionnaire contenant les langues et leurs URL associées si tout se passe bien, soit un dictionnaire vide si une erreur se produit lors de la récupération des liens.

In [4]:
def extract_paragraphs(driver, url: str) -> tuple:
    """
    Extrait les paragraphes d'un article Wikipédia

    Parameters:
    driver (WebDriver): Le driver Selenium
    url (str): L'URL de l'article Wikipédia

    Returns:
    str: Les paragraphes de l'article
    str: Le titre de l'article
    """
    driver.get(url)
    
    paragraphs = ""

    # Extract the first paragraph of the article
    try:
        title = os.path.basename(urllib.parse.urlparse(url).path)
        elements = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/p')

        print(f"Found {len(elements)} paragraphs for {url} : {[el.text[:30] for el in elements]}")

        for el in elements:
            if el.text:
                paragraphs += el.text + "\n"
            else:
                continue

    except:
        print(f"Could not find paragraph for {url}")

    return paragraphs, title

Je définis ici une fonction appelée `extract_paragraphs` qui prend deux paramètres : `driver` et `url`.

Voici ce que je fais dans cette fonction :

1. Je commence par charger l'URL donnée dans le navigateur à l'aide de `driver.get(url)`.

2. Je déclare une variable `paragraphs` que j'initialise avec une chaîne de caractères vide. Cela servira à stocker tous les paragraphes de texte que je vais extraire.

3. Je tente ensuite d'extraire le premier paragraphe de l'article. Pour cela, je procède comme suit :

   a. J'extrais le titre de la page à partir de l'URL en utilisant les fonctions `os.path.basename` et `urllib.parse.urlparse` pour obtenir uniquement la partie finale de l'URL, qui correspond généralement au titre de l'article.

   b. Je recherche dans le document tous les éléments qui correspondent au chemin XPath spécifié. Ce chemin cible les paragraphes situés dans une division particulière de la page identifiée par l'identifiant `mw-content-text`.

   c. J'affiche un message indiquant combien de paragraphes j'ai trouvés pour l'URL donnée. Pour chaque élément trouvé, je montre les 30 premiers caractères du texte de cet élément.

   d. Je parcours ensuite chaque élément trouvé. Si l'élément contient du texte, je l'ajoute à la variable `paragraphs` avec un saut de ligne à la fin. Si l'élément ne contient pas de texte, je continue simplement sans l'ajouter.

4. Si une exception se produit pendant l'extraction (par exemple, si le chemin XPath est incorrect ou si la page ne se charge pas correctement), j'affiche un message d'erreur indiquant que je n'ai pas pu trouver de paragraphe pour l'URL donnée.

5. Enfin, je retourne un tuple contenant les paragraphes extraits et le titre de l'article. Cela permet à celui qui utilise cette fonction de recevoir à la fois le texte extrait et le titre de la page à partir de laquelle le texte a été extrait.

In [5]:
def write_txt(data_folder: str, lang: str, url: str, paragraph: str, file_counts: dict) -> None:
    """
    Écrit un paragraphe récupéré dans un fichier texte

    Parameters:
    data_folder (str): Le chemin du dossier où écrire le fichier texte
    lang (str): La langue de l'article
    url (str): L'URL de l'article
    paragraph (str): Le paragraphe à écrire
    file_counts (dict): Un dictionnaire avec les langues en clés et le nombre de fichiers écrits en valeurs

    Returns:
    None
    """
    # Parse the URL to extract the article title, which is the part after the last '/'
    parsed_url = urllib.parse.urlparse(url)
    article_title = os.path.basename(parsed_url.path)
    
    # URL decode the article title (e.g., replace '%E1%83%90' with actual characters)
    article_title = urllib.parse.unquote(article_title)
    
    # Replace any remaining invalid file name characters (like '/') in the title with '_'
    valid_article_title = ''.join(char if char.isalnum() else '_' for char in article_title)
    
    # Ensure the data folder exists
    os.makedirs(data_folder, exist_ok=True)
    
    # Generate the filename and write the paragraph to the file
    filename = f"{lang}_{valid_article_title}.txt"
    file_path = os.path.join(data_folder, filename)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(paragraph)
        print(f"Successfully wrote {file_path}\n")
    file_counts[lang] += 1


Je définis une fonction nommée `write_txt` qui prend cinq paramètres : `data_folder`, `lang`, `url`, `paragraph`, et `file_counts`. Ces paramètres me servent respectivement à spécifier où sauvegarder les fichiers, la langue de l'article, l'URL de l'article, le paragraphe à écrire, et un dictionnaire pour suivre combien de fichiers j'ai écrits pour chaque langue.

Voici comment je procède dans cette fonction :

1. **Extraction du titre de l'article depuis l'URL** : 
   - Je parse l'URL pour isoler la partie qui me sert de titre, qui est après le dernier '/' dans le chemin de l'URL.
   - Je décode le titre pour convertir des séquences d'échappement URL (comme '%E1%83%90') en caractères lisibles.

2. **Nettoyage du titre** :
   - Je remplace tout caractère du titre qui n'est pas alphanumérique par un underscore ('_') pour m'assurer que le titre peut être utilisé comme nom de fichier sans causer d'erreur.

3. **Préparation du dossier de données** :
   - Je m'assure que le dossier où je veux sauvegarder les fichiers existe. S'il n'existe pas, je le crée avec `os.makedirs`, et l'option `exist_ok=True` me permet de ne pas générer d'erreur si le dossier existe déjà.

4. **Écriture du paragraphe dans un fichier** :
   - Je génère le nom du fichier en utilisant la langue et le titre de l'article nettoyé, et j'ajoute l'extension '.txt'.
   - Je combine le dossier de données et le nom du fichier pour créer le chemin complet du fichier.
   - J'ouvre le fichier en mode écriture (`'w'`) avec encodage UTF-8, et j'écris le paragraphe dedans. Je termine par afficher un message pour confirmer que l'écriture a réussi.

5. **Incrémentation du compteur de fichiers** :
   - J'augmente de 1 le compteur pour la langue correspondante dans le dictionnaire `file_counts` pour suivre combien de fichiers ont été écrits pour chaque langue.

La fonction `write_txt` n'a pas de valeur de retour. Elle effectue son travail d'écriture en sauvegardant les paragraphes dans des fichiers spécifiques tout en organisant et en comptant combien de fois elle le fait pour chaque langue.

In [6]:
def main():
    driver = webdriver.Chrome()
    data_folder = "../../../data/raw/"
    languages = ['ar', 'fr', 'en', 'de', 'es', 'ja', 'ko', 'ru', 'zh']
    file_counts = {lang: 0 for lang in languages}

    base_urls = get_random_page_urls(driver, languages)
    for lang in languages:
        while file_counts[lang] < 40:
            current_url = base_urls[lang]
            paragraphs, article_title = extract_paragraphs(driver, current_url)
            if paragraphs:
                write_txt(data_folder, lang, article_title, paragraphs, file_counts)
            
            links = get_interlanguage_urls(driver, current_url)
            for link_lang, link_url in links.items():
                if link_lang in languages and file_counts[link_lang] < 40:
                    link_paragraphs, title = extract_paragraphs(driver, link_url)
                    if link_paragraphs:
                        write_txt(data_folder, link_lang, title, link_paragraphs, file_counts)

            # Refreash the base URL for the current language
            new_urls = get_random_page_urls(driver, [lang])
            base_urls[lang] = new_urls[lang]
            
    driver.quit()



J'ai défini une fonction nommée `main` qui orchestre l'extraction et l'enregistrement de textes à partir de différentes éditions linguistiques de Wikipedia. Voici comment je m'y prends :

1. **Initialisation du pilote du navigateur** :
   - J'initialise un pilote Chrome en utilisant `webdriver.Chrome()`. Cela me permet d'ouvrir et de manipuler un navigateur Chrome directement depuis mon script.

2. **Définition du dossier de données et des langues** :
   - Je définis le chemin vers le dossier où je souhaite enregistrer les fichiers extraits. Je spécifie également une liste de codes de langue pour les éditions de Wikipedia que je souhaite explorer.

3. **Initialisation des compteurs de fichiers** :
   - Je crée un dictionnaire `file_counts` pour suivre le nombre de fichiers que j'ai écrits pour chaque langue, en initialisant tous les compteurs à zéro.

4. **Récupération des URL initiales** :
   - J'utilise la fonction `get_random_page_urls` pour obtenir une URL de page aléatoire pour chaque langue listée. Cela me donne un point de départ pour chaque langue.

5. **Traitement pour chaque langue** :
   - Pour chaque langue, je continue à extraire et enregistrer des paragraphes tant que je n'ai pas atteint 20 fichiers pour cette langue. Je récupère l'URL actuelle pour la langue en question et j'extrait des paragraphes et le titre de l'article à partir de cette URL.

6. **Écriture des paragraphes** :
   - Si des paragraphes sont extraits, je les enregistre en utilisant la fonction `write_txt`.

7. **Exploration des liens interlangues** :
   - J'utilise `get_interlanguage_urls` pour récupérer des liens vers d'autres éditions linguistiques de la même page Wikipedia. Pour chaque lien valide qui pointe vers une langue que je suis en train de traiter et où je n'ai pas encore atteint la limite de 20 fichiers, j'extrait et j'enregistre les paragraphes.

8. **Rafraîchissement des URL** :
   - Après avoir traité les liens interlangues, je rafraîchis l'URL de base pour la langue actuelle en récupérant une nouvelle URL aléatoire. Cela me permet de continuer à extraire de nouveaux contenus sans boucler sur les mêmes pages.

9. **Nettoyage** :
   - Une fois que j'ai fini de traiter toutes les langues, je ferme le pilote du navigateur avec `driver.quit()` pour libérer les ressources.

Cette fonction `main` organise donc systématiquement l'extraction de contenus multilingues à partir de Wikipedia, les sauvegarde localement, et assure que le nombre de fichiers pour chaque langue atteint un seuil spécifié. Cela me permet de construire une collection diversifiée de textes pour différents usages, comme l'entraînement de modèles de traitement de langues.

In [7]:
main()

Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D8%AA%D9%8A%D9%88_%D9%84%D9%8A%D9%86 : ['ماتيو لين (بالفرنسية: Mathieu ']
Successfully wrote ../../../data/raw/ar_ماتيو_لين.txt

Found 37 paragraphs for https://en.wikipedia.org/wiki/Mathieu_Laine : ['Mathieu Laine (born April 9, 1', 'He is the founder and head of ', 'He is an affiliate professor a', 'Son and grandson of doctors, M', 'He holds a certificate in the ', 'Mathieu Laine taught business ', 'In 2018, he was appointed Affi', 'Mathieu Laine was a corporate ', 'Mathieu Laine created Altermin', 'Mathieu Laine became an adviso', 'In 2014, Mathieu Laine moved t', 'Since the launch of Altermind,', 'He advises several executives ', 'According to an article in Les', 'Among Altermind’s achievements', 'Another mission relayed in the', "Altermind's partners include A", 'In July 2020, Altermind announ', 'Mathieu Laine has been a regul', 'In April 2005, after having be', 'In 2002, Laine published an op', 'Mathieu Laine is in

2024-05-05 10:57:49,091 - ERROR - Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D9%86%D9%88%D8%A7%D9%84_%D9%85%D9%87%D9%86%D9%8A


Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D8%AC%D9%88%D9%86_%D8%B3%D8%AA%D8%B1%D9%8A%D9%86%D8%AC : ['جون سترينج هو سياسي كندي، ولد ', 'ولد في إيست كيلبرايد، اسكتلندا']
Successfully wrote ../../../data/raw/ar_جون_سترينج.txt

Found 4 paragraphs for https://en.wikipedia.org/wiki/John_Strange_(Upper_Canada_politician) : ['John Strange (September 2, 178', 'He was born in East Kilbride, ', 'Strange was a father of five, ', '']
Successfully wrote ../../../data/raw/en_John_Strange__Upper_Canada_politician_.txt

Found 22 paragraphs for https://ar.wikipedia.org/wiki/%D8%B4%D8%B0%D9%89_%D8%AD%D8%B3%D9%88%D9%86 : ['شذى أمجد الحسون (مواليد 3 مارس', 'ولدت بمدينة الدار البيضاء بالم', 'لديها أخ اصغر منها اسمه "وائل"', 'منذ الطفولة كان هدف شذى حسون ه', 'في عام 2006 كانت الصدفة حليفة ', 'حينها تميزت شذى عن باقي الطالب', 'كانت عدة قنوات محلية بضمنها قن', 'كما قال بعض المراقبين أن شذى ق', 'منذ وجودها في الأكاديمية فازت ', 'لتكرر التجربة للمرة الثانية عل', 'لتقوم نفس الشركة بأستغلال شعبي', 'شار

2024-05-05 11:00:15,473 - ERROR - Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AC%D8%AD%D8%A7%D9%88%D8%B1_(%D8%A7%D9%84%D9%82%D9%81%D8%B1)


Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%AF%D9%8A%D9%81_%D8%AF%D9%88%D9%86%D8%A7%D9%84%D8%AF%D8%B3%D9%88%D9%86_(%D9%84%D8%A7%D8%B9%D8%A8_%D9%83%D8%B1%D8%A9_%D9%82%D8%AF%D9%85) : ['ديف دونالدسون (بالإنجليزية: Da']
Successfully wrote ../../../data/raw/ar_ديف_دونالدسون__لاعب_كرة_قدم_.txt

Found 3 paragraphs for https://en.wikipedia.org/wiki/Dave_Donaldson_(footballer,_born_1941) : ['', 'David Donaldson (born 28 Decem', '']
Successfully wrote ../../../data/raw/en_Dave_Donaldson__footballer__born_1941_.txt

Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%BA%D8%B1%D9%81_(%D8%AD%D8%B6%D8%B1%D9%85%D9%88%D8%AA) : ['الغرف هي إحدى قرى الجمهورية ال', '']
Successfully wrote ../../../data/raw/ar_الغرف__حضرموت_.txt



...

Found 2 paragraphs for https://zh.wikipedia.org/wiki/%E6%B3%B0%E5%9C%8B%E6%9D%91%E6%8E%A7%E8%82%A1 : ['泰國村控股有限公司，簡稱泰國村控股（英語：Thai Vill', '總部位於Blk 1002 Tai Seng Ave #01-']
Successfully wrote ../../../data/raw/zh_泰國村控股.txt



2024-05-05 11:23:45,789 - ERROR - Timeout when trying to find interlanguage links for https://zh.wikipedia.org/wiki/%E6%B3%B0%E5%9C%8B%E6%9D%91%E6%8E%A7%E8%82%A1


Found 4 paragraphs for https://zh.wikipedia.org/wiki/%E9%87%91%E5%8D%AB%E4%B8%9C%E7%AB%99 : ['金山卫东站又名金卫东站，位于中国上海市金山区沪杭公路石化汽车', '2002年时，该站尚有往来上海南站的专1、专4、专5、专8、', '2002年10月21日，原上海南站到发的4对市郊列车转到属临', '至2003年6月18日，4对来往于梅陇站和金卫东站的市郊列车']
Successfully wrote ../../../data/raw/zh_金卫东站.txt



2024-05-05 11:24:17,855 - ERROR - Timeout when trying to find interlanguage links for https://zh.wikipedia.org/wiki/%E9%87%91%E5%8D%AB%E4%B8%9C%E7%AB%99


Found 3 paragraphs for https://zh.wikipedia.org/wiki/%E6%B8%B8%E6%BC%A0%E8%9E%B3%E5%B1%9E : ['', '游漠螳属（学名：Eremoplana）为谷螳科的一个属。', '本属包括以下物种：[1]']
Successfully wrote ../../../data/raw/zh_游漠螳属.txt

Found 8 paragraphs for https://zh.wikipedia.org/wiki/%E5%B0%BC%E5%8F%A4%E6%8B%89%C2%B7%E6%99%AE%E5%B0%94%E7%83%AD%E7%93%A6%E5%B0%94%E6%96%AF%E5%9F%BA : ['尼古拉·米哈伊洛维奇·普尔热瓦尔斯基（俄語：Никола́й', '普尔热瓦尔斯基出身於白俄羅斯一個貴族家庭，曾在波蘭華沙軍事學', '虽然他在探險生涯中从未实现前往西藏首府拉薩這一目标，但是他在', '据戴维·希梅尔彭尼克·范德奥伊评价，普尔热瓦尔斯基的中亚著作', '普尔热瓦尔斯基以及其他当代探险家，包括斯文·赫定、弗朗西斯·', '普尔热瓦尔斯基的偏见也延伸到了除华人以外的其他亚洲人身上，他', '普尔热瓦尔斯基不仅蔑视中国主体民族，他还认为西藏、突厥斯坦和', '普尔热瓦尔斯基建议俄罗斯煽动中国这些地区的佛教徒和穆斯林反抗']
Successfully wrote ../../../data/raw/zh_尼古拉_普尔热瓦尔斯基.txt

Found 7 paragraphs for https://zh.wikipedia.org/wiki/%E8%89%BE%E6%88%88%E5%A5%88 : ['艾戈奈（法語：Aigonnay，法语发音：[ɛɡɔnɛ]）是', '艾戈奈（46°19\'47"N, 0°15\'30"W）面积14', '与艾戈奈接壤的市镇（或旧市镇、城区）包括：穆贡、普拉耶、罗芒', '艾戈奈的时区为UTC+01:00、UTC+02:00（夏令时', '艾戈奈的邮政编码为79370，INSEE市镇编码为79004', '艾戈奈所属的省级选区为贝勒河畔塞勒县。', '艾戈奈于2018年1月1日时的人口数量为634人。']
Successfully wr